# Map-making with Python
### By Maggie Orton and Alex Cao
### April 19, 2017
### <a href="http://cscar.research.umich.edu">CSCAR</a> at The University of Michigan  

Please fill out the workshop sign-in <a href="https://goo.gl/forms/kLpPDHKlwZIV5ZpD2">here</a>

This workshop will cover how to make a few basic types of maps using matplotlib in Python. The types of maps covered are:
1. Choropleth
2. Dot map
3. Proportional dot map
4. Isopleth

#### Note:  
to install geopandas on a Mac with Anaconda:  
$ conda install -c https://conda.anaconda.org/ioos geopandas

In [1]:
#% matplotlib inline

# Choropleth

Importing required modules and limiting pandas output to 12 rows:

In [2]:
import pandas as pd
import time
import requests
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.collections import PolyCollection
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import numpy as np
from itertools import chain

pd.options.display.max_rows = 12

### Generating GeoDataFrame of MI counties

In [3]:
# Read GeoJSON from data portal and convert to a dataframe
R = requests.get(r'http://gis-michigan.opendata.arcgis.com/datasets/67a8ff23b5f54f15b7133b8c30981441_0.geojson')
R.raise_for_status()
county_geojson= R.json()

list_counties = []
for county in county_geojson['features']:
    fips = county['properties']['FIPSNUM']
    name = county['properties']['NAME']
    geomtype = county['geometry']['type']
    coords = county['geometry']['coordinates']
    list_counties.append((fips,name,geomtype,coords))

counties = pd.DataFrame(list_counties, columns=['fips','NAME','geomtype','coords'])
counties

,fips,NAME,geomtype,coords
0,1,Alcona,Polygon,"[[[-83.88711741272411, 44.553854376313986], [-..."
1,3,Alger,MultiPolygon,"[[[[-87.11602141361914, 46.27725994774438], [-..."
2,5,Allegan,Polygon,"[[[-85.54342616590016, 42.42905407573252], [-8..."
3,7,Alpena,MultiPolygon,"[[[[-83.34340013941748, 44.88554342308513], [-..."
4,9,Antrim,MultiPolygon,"[[[[-84.84877327763279, 44.93221680302625], [-..."
5,11,Arenac,MultiPolygon,"[[[[-83.75550427206387, 43.99347015001873], [-..."
...,...,...,...,...
77,155,Shiawassee,Polygon,"[[[-83.92586020762252, 42.844389287953845], [-..."
78,157,Tuscola,MultiPolygon,"[[[[-83.2022671025338, 43.32524973578166], [-8..."
79,159,Van Buren,Polygon,"[[[-86.2230971180473, 42.19231680829333], [-86..."


### Loading choropleth snow crash data

In [4]:
# Load choropleth data
fatals = pd.read_csv('snow_crashes.csv', sep='\t')
fatals.set_index('County', inplace=True)
# convert string to integers
fatals = fatals.applymap(lambda x: int(str(x).replace(',','')))
fatals['snow_crashes'] = fatals['Total']

# merge snow crash data with geodataframe
df = counties.merge(pd.DataFrame(fatals['snow_crashes']), how='inner', left_on='NAME', right_index=True)
df.head(2)

,fips,NAME,geomtype,coords,snow_crashes
0,1,Alcona,Polygon,"[[[-83.88711741272411, 44.553854376313986], [-...",21
1,3,Alger,MultiPolygon,"[[[[-87.11602141361914, 46.27725994774438], [-...",37


### Formatting colors and labels

In [5]:
# function assigns x a color based on minimums set in bins (inclusive)
# requires: same number of bins and colors
def assign_color(x, colors, bins):
    bincount = len(bins)
    counter = 1
    while (counter < bincount):
        if x >= bins[-counter]:
            return colors[-counter]
        counter = counter + 1
    return colors[0]

In [6]:
# example usage of assign_color: assigning value '5' to blue in 5-9 bin
x = 6
assign_color(x, ['green', 'blue', 'red'], [0, 5, 10])

'blue'

In [7]:
# setting up choropleth parameters
colors = ['#edf8e9','#bae4b3','#74c476','#31a354','#006d2c']
bins = [0,100,200,400,800]
labels = []
# formatting labels for bins
for i in range(len(bins)):
    if i < len(bins)-1:
        label = '{} - {}'.format(bins[i]+1,bins[i+1])
    else:
        label = '{}+'.format(bins[i])
    labels.append(label)
labels

['1 - 100', '101 - 200', '201 - 400', '401 - 800', '800+']

In [8]:
df['color'] = df['snow_crashes'].apply(assign_color, args=(colors,bins))
df[['NAME', 'snow_crashes', 'color']].head(4)

,NAME,snow_crashes,color
0,Alcona,21,#edf8e9
1,Alger,37,#edf8e9
2,Allegan,429,#31a354
3,Alpena,52,#edf8e9


### Plotting choropleth

In [9]:
inches = 12 # Approximately 1200 x 1200 pixels
fig = plt.figure(2, figsize=(inches,inches), frameon=True)
ax = plt.gca()
# add blue for the great lakes (and land)
fig.patch.set_facecolor('#9bbff4')

In [10]:
# create a nested list of polygons, vertices, and xy-coordinates 
mi = []
for i in range(df.shape[0]):
    geom_type = df.at[i,'geomtype']
    poly_xy = df.at[i,'coords']
    if geom_type == 'Polygon':
        mi.append(poly_xy[0])
    elif geom_type == 'MultiPolygon':
        for poly in poly_xy:
            mi.append(poly[0])

In [11]:
# Make a polygon collection and plot it with colour column
coll = PolyCollection(mi, facecolors=df['color'], edgecolors='white')
ax.add_collection(coll)
# this line is necessary for collection to prevent crashing
ax.autoscale_view()

In [12]:
#%% Calculate Approximate Centre
xyc = []
for i in range(df.shape[0]):
    geom_type = df.at[i,'geomtype']
    poly_xy = df.at[i,'coords']
    if geom_type == 'Polygon':
        X,Y = zip(*poly_xy[0])
    elif geom_type == 'MultiPolygon':
        listx, listy = [],[]
        for poly in poly_xy:
            x,y = zip(*poly[0])
            listx.append(x)
            listy.append(y)
        X = list(chain.from_iterable(listx))
        Y = list(chain.from_iterable(listy))
    dfX = pd.Series(X)
    dfY = pd.Series(Y)
    xc = 0.5*(dfX.max()+dfX.min())
    yc = 0.5*(dfY.max()+dfY.min())
    xyc.append([xc,yc])        
centroid = pd.Series(xyc)
df['centroid'] = centroid
df

,fips,NAME,geomtype,coords,snow_crashes,color,centroid
0,1,Alcona,Polygon,"[[[-83.88711741272411, 44.553854376313986], [-...",21,#edf8e9,"[-83.578763451, 44.68422868]"
1,3,Alger,MultiPolygon,"[[[[-87.11602141361914, 46.27725994774438], [-...",37,#edf8e9,"[-86.4904581968, 46.4246261354]"
2,5,Allegan,Polygon,"[[[-85.54342616590016, 42.42905407573252], [-8...",429,#31a354,"[-85.9084427462, 42.593882269]"
3,7,Alpena,MultiPolygon,"[[[[-83.34340013941748, 44.88554342308513], [-...",52,#edf8e9,"[-83.5405464163, 45.0318546931]"
4,9,Antrim,MultiPolygon,"[[[[-84.84877327763279, 44.93221680302625], [-...",75,#edf8e9,"[-85.1452156743, 45.0100784647]"
5,11,Arenac,MultiPolygon,"[[[[-83.75550427206387, 43.99347015001873], [-...",30,#edf8e9,"[-83.7967836299, 44.0369963508]"
...,...,...,...,...,...,...,...
77,155,Shiawassee,Polygon,"[[[-83.92586020762252, 42.844389287953845], [-...",123,#bae4b3,"[-84.1452094224, 42.9542881717]"
78,157,Tuscola,MultiPolygon,"[[[[-83.2022671025338, 43.32524973578166], [-8...",101,#bae4b3,"[-83.4017620453, 43.4751737963]"
79,159,Van Buren,Polygon,"[[[-86.2230971180473, 42.19231680829333], [-86...",357,#74c476,"[-86.0637764782, 42.2450020191]"


In [13]:
# Plot county text
fs = 7
for i in range(df.shape[0]):
    xy = df.at[i,'centroid']
    xc = xy[0]
    yc = xy[1]
    name = df.at[i,'NAME']
    # xc, yc are in same units as axis (lat/long coords)
    plt.text(xc, yc, name, fontsize=fs, ha='center', weight='bold')        

In [14]:
# Draw legend
alpha = .8
swatches = []
for color, label in zip(colors,labels):
    swatches.append(mpatches.Patch(color=color, label=label, alpha=alpha))
plt.legend(handles=swatches, bbox_to_anchor=(0.4,0.3), frameon=False)

In [15]:
# Plot title
# percentage of figure
plt.figtext(0.33,0.32, 'Vehicle Crashes\nIn Snowy Conditions', fontsize=fs+3, weight='bold', ha='center', va='bottom')

In [16]:
# Format figure and save
plt.axis('off') #hides the axes from showing on the graph 
plt.tight_layout() #minimizes the spacing around the outside of the plot
# can be saved in pixel format (jpg, png) or vector (svg)
fig.savefig("mi_choropleth.png", facecolor=fig.get_facecolor(), bbox_inches='tight')

# Dot Map

### Creating new map to add dots to

In [17]:
plt.gcf().clear()
fig2 = plt.figure(2, figsize=(inches,inches), frameon=True)
ax2 = plt.gca()
# add blue for the great lakes (and land)
fig2.patch.set_facecolor('#9bbff4')

In [18]:
# Make a polygon collection and plot it
coll = PolyCollection(mi, facecolors='#edf8e9', edgecolors='white')
ax2.add_collection(coll)
# this line is necessary for collection to prevent crashing
ax2.autoscale_view()

In [19]:
# Plot title
plt.figtext(0.5,0.9, 'Snowmobile Crashes', fontsize=fs+20, weight='bold', ha='center', va='bottom')

### Reading in snowmobile crash data

In [20]:
snowmobiles = pd.read_table("snowmobile_crashes.txt", sep = "\t", usecols = ["Crash Longitude", "Crash Latitude"])
print(snowmobiles[:3])

   Crash Longitude  Crash Latitude
0       -85.800058       46.180844
1       -84.789822       45.440444
2       -86.740536       46.321993


### Plotting circles
more information on different marker styles available <a href="https://matplotlib.org/api/markers_api.html#module-matplotlib.markers">here</a>

In [21]:
ax2.scatter(snowmobiles['Crash Longitude'], snowmobiles['Crash Latitude'], c = 'maroon', alpha = .6, marker = 'p')
for i, [lat, lon] in enumerate(zip(snowmobiles['Crash Latitude'], snowmobiles['Crash Longitude'])):
    plt.text(lon, lat, i)

In [22]:
# Format figure and save
plt.axis('off')
plt.tight_layout()
fig2.savefig("snowmobile_crashes.png", facecolor=fig2.get_facecolor(), bbox_inches='tight')

### Practice
See if you can change the scatterplot marker to a different shape

# Proportional Dot Map

### Creating new map to add dots to

In [23]:
plt.gcf().clear()
fig3 = plt.figure(2, figsize=(inches,inches), frameon=True)
ax3 = plt.gca()
# add blue for the great lakes (and land)
fig3.patch.set_facecolor('#9bbff4')

In [24]:
# Make a polygon collection and plot it
coll = PolyCollection(mi, facecolors='#edf8e9', edgecolors='white')
ax3.add_collection(coll)
# this line is necessary for collection to prevent crashing
ax3.autoscale_view()
plt.axes().set_aspect('equal', 'datalim')

In [25]:
# Plot title
plt.figtext(0.5, .9, 'Top 20 Deer Crashes', fontsize=fs+20, weight='bold', ha='center', va='bottom')

### Reading in deer crash data

In [26]:
deer = pd.read_csv("deer_in_the_city.txt").sort_values(by = ["Total"])
# take top 20
deer = deer[-20:]
deer

,city,Total,K,ABC,PDO,Lat,Lon
19,Burton,40,0,1,39,42.999472,-83.616342
18,Norton Shores,41,0,2,39,43.168904,-86.263946
17,Sterling Heights,44,0,3,41,42.580312,-83.030203
16,Southfield,44,0,6,38,42.473369,-83.221873
15,Wyoming,46,0,0,46,42.913360,-85.705309
14,East Lansing,55,0,3,52,42.736979,-84.483865
...,...,...,...,...,...,...,...
5,Ann Arbor,90,0,4,86,42.280826,-83.743038
4,Farmington Hills,95,0,4,91,42.498994,-83.367717
3,Battle Creek,116,0,6,110,42.321152,-85.179714


In [27]:
deer["Total"] = pd.to_numeric(deer["Total"])

### Plotting circles

In [28]:
for lat, lon, num in zip(deer['Lat'], deer['Lon'], deer['Total']):
    circ = plt.Circle((lon, lat), radius = num / 500, color = 'maroon', alpha = .3, fill = True)
    ax3.add_artist(circ)

In [29]:
# Format figure and save
plt.axis('off')
plt.tight_layout()
fig3.savefig("top_20_crashes.png", facecolor=fig2.get_facecolor(), bbox_inches='tight')

# Isopleth

In [34]:
plt.gcf().clear()

# read in temperature
temps = pd.read_csv("mi_temperature.txt")
lats = temps["latitude"].drop_duplicates().sort_values()
lons = temps["longitude"].drop_duplicates().sort_values()
Y, X = np.meshgrid(lats, lons) # creates numpy arrays of all X/Y combos
temps = temps.sort_values(by = ["longitude", "latitude"])

temps

,maxTemp,minTemp,wind,longitude,latitude
0,28.55,15.76,19.90,-90.3,46.6
1,28.45,15.31,21.38,-90.2,46.5
2,28.26,15.39,21.06,-90.2,46.6
3,29.20,15.80,20.93,-90.1,46.4
4,28.22,15.05,21.97,-90.1,46.5
5,28.46,15.36,21.53,-90.1,46.6
...,...,...,...,...,...
1700,44.72,35.78,18.24,-82.6,43.6
1701,50.80,37.59,19.39,-82.5,42.7
1702,50.34,38.02,18.96,-82.5,42.8


In [35]:
# create dataframe of all lat/lon combos to merge temps with
latlons = pd.DataFrame(columns = ["longitude", "latitude"])
latlons["longitude"] = list(lons) * len(lats)
latlons["latitude"] = list(np.repeat(lats, len(lons)))
latlons = latlons.sort_values(by = ["longitude", "latitude"])
# merge temps with new dataframe so all possible lat/lon combos are present
temps = pd.merge(temps, latlons, how = 'right', on = ["latitude", "longitude"], sort = True)
temps = temps.fillna(np.nanmin(list(temps["maxTemp"])) - 2) # fill non-land values with lowest temp - 2
# create new Z array from sorted maxTemp column
Z = temps.pivot(index = 'longitude', columns = 'latitude', values = 'maxTemp')
temps

,maxTemp,minTemp,wind,longitude,latitude
0,22.87,22.87,22.87,-90.3,41.7
1,22.87,22.87,22.87,-90.2,41.7
2,22.87,22.87,22.87,-90.1,41.7
3,22.87,22.87,22.87,-90.0,41.7
4,22.87,22.87,22.87,-89.9,41.7
5,22.87,22.87,22.87,-89.8,41.7
...,...,...,...,...,...
4813,22.87,22.87,22.87,-83.0,48.1
4814,22.87,22.87,22.87,-82.9,48.1
4815,22.87,22.87,22.87,-82.8,48.1


In [36]:
print(X)
print(Y)
# plot and save figure
fig4 = plt.figure(figsize=(inches,inches), frameon=False)
plt.axes().set_aspect('equal', 'datalim')
CS = plt.contour(X, Y, Z, 20) # will use 20 isopleth cutoffs
plt.clabel(CS, inline = 1, fontsize = 10)
plt.axis('off')
fig4.savefig("isopleth.png")

[[-90.3 -90.3 -90.3 ..., -90.3 -90.3 -90.3]
 [-90.2 -90.2 -90.2 ..., -90.2 -90.2 -90.2]
 [-90.1 -90.1 -90.1 ..., -90.1 -90.1 -90.1]
 ..., 
 [-82.7 -82.7 -82.7 ..., -82.7 -82.7 -82.7]
 [-82.6 -82.6 -82.6 ..., -82.6 -82.6 -82.6]
 [-82.5 -82.5 -82.5 ..., -82.5 -82.5 -82.5]]
[[ 41.7  41.8  41.9 ...,  47.9  48.   48.1]
 [ 41.7  41.8  41.9 ...,  47.9  48.   48.1]
 [ 41.7  41.8  41.9 ...,  47.9  48.   48.1]
 ..., 
 [ 41.7  41.8  41.9 ...,  47.9  48.   48.1]
 [ 41.7  41.8  41.9 ...,  47.9  48.   48.1]
 [ 41.7  41.8  41.9 ...,  47.9  48.   48.1]]


### Practice
Try using the 'wind' column instead to make a different plot  

Try changing the number of isopleth cutoffs  

Try filling the blank areas with 0 instead of lowest - 2